# 实验题目

使用决策树完成信誉度分类任务

*注：由于机器学习相关实验需要大量用到jupyter notebook环境，为了带来更清晰的报告结构、更好的视觉体验，接下来的实验报告，我大部分会使用LaTeX编写。*

# 实验原理

## 决策树算法

决策树是最经典的机器学习算法之一，主要用于分类和回归任务。其构建过程基于递归分割：

- 从根节点开始，选择一个最优特征进行数据分割，生成子节点；

- 在每个子节点上重复此过程，直到满足停止条件，如节点纯度达到一定程度、达到预设的最大深度等。

## 特征划分方法

上面说过，决策树就是不断分割特征的过程。这里介绍两种常见的分割算法。

### ID3算法

ID3 (Iterative Dichotomiser 3) 算法使用信息增益作为划分特征的标准。

给定训练集 D 和特征 A，信息增益 $\text{Gain}(D, A)$ 定义为：

$$
\text{Gain}(D, A) = \text{Entropy}(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \text{Entropy}(D_v)
$$

其中，$\text{Entropy}(D)$ 是数据集 D 的熵，计算公式为：

$$
\text{Entropy}(D) = -\sum_{k} p_k \log_2 p_k
$$

$p_k$ 是类别 k 在数据集 D 中的比例。

### C4.5算法

C4.5 算法改进了 ID3，通过使用信息增益率来克服 ID3 偏好选择具有更多值的特征的问题。

信息增益率定义为信息增益和特征 A 的固有信息（分割信息）之比：

$$
\text{Gain\_ratio}(D, A) = \frac{\text{Gain}(D, A)}{\text{Split\_Info}(D, A)}
$$

其中，分割信息 $\text{Split\_Info}(D, A)$ 的计算公式为：

$$
\text{Split\_Info}(D, A) = -\sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \log_2 \frac{|D_v|}{|D|}
$$

## 剪枝方法

剪枝是决策树算法中用于减少过拟合和提高模型泛化能力的重要技术。

### 预剪枝

预剪枝在决策树完全生成前进行。预剪枝提前停止某些分支的生长，防止过拟合。这可以通过设定最大深度、最小样本数或最小信息增益等参数来实现。

### 后剪枝

后剪枝在决策树完全生成后进行。后剪枝从树的底部开始，尝试去除某些子节点（用叶节点替换），并检验这种简化是否能提高测试集上的准确率，从而优化决策树的结构。

# 实验过程

我们开始进入编码环节。首先导入必须的包。

In [1]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

## 准备数据

首先我们读取题目给定的数据，并进行简单探索。

In [2]:
df = pd.read_csv('data/DT_data.csv')

In [3]:
df.head()

,Undergrad,MaritalStatus,TaxableIncome,WorkExperience,Urban
0,NO,Single,68833,10,YES
1,YES,Divorced,33700,18,YES
2,NO,Married,36925,30,YES
3,YES,Single,50190,15,YES
4,NO,Married,81002,28,NO


接下来，我们开始对数据进行预处理。

根据题目要求，我们首先根据原数据中的`TaxableIncome`列，预处理出`Label`列。

我们定义，当`TaxableIncome>30000`时，信用为好，即`Label=1`，反之`Label=0`。

In [4]:
df['Label'] = (df['TaxableIncome'] > 30000).astype(np.int32)

In [5]:
df.drop('TaxableIncome', axis=1, inplace=True)

In [6]:
df.head()

,Undergrad,MaritalStatus,WorkExperience,Urban,Label
0,NO,Single,10,YES,1
1,YES,Divorced,18,YES,1
2,NO,Married,30,YES,1
3,YES,Single,15,YES,1
4,NO,Married,28,NO,1


除了预处理出`Label`列外，考虑到决策树的特征输入类型应该为`int`而非`string`，我们还需要将原数据中的`Undergrad`列等进行预处理。

以`Undergrad`列为例。我们需要将`NO`映射到`0`，`YES`映射为`1`。

事实上，`sklearn`库的决策树是可以接受`string`作为输入类型的。但是考虑到我们这里是自己手写决策树，最好还是处理为`int`更为方便。

In [7]:
df['Undergrad']     = df['Undergrad'].map({'NO': 0, 'YES': 1})
df['MaritalStatus'] = df['MaritalStatus'].map({'Single': 0, 'Divorced': 1, 'Married': 2})
df['Urban']         = df['Urban'].map({'NO': 0, 'YES': 1})

我们接下来处理`WorkExperience`列。这一列原本的数据是连续的整数，但在决策树中，输入特征应该为类别而非具体的数值。

因此，我们对`WorkExperience`列以 $5$ 为区间进行划分，将原本连续的的 $0\sim 40$ 工作经验，划分为离散的 $0\sim 8$ 类。

In [8]:
bins   = [0, 5, 10, 15, 20, 25, 30, 35, 40]
labels = [0, 1, 2 , 3 , 4 , 5 , 6 , 7 , 8 ]

In [9]:
df['WorkExperience'] = np.digitize(df['WorkExperience'], bins=bins, right=True)

In [10]:
df.head()

,Undergrad,MaritalStatus,WorkExperience,Urban,Label
0,0,0,2,1,1
1,1,1,4,1,1
2,0,2,6,1,1
3,1,0,3,1,1
4,0,2,6,0,1


直到现在，我们已经完成了数据的预处理。我们顺手保存一下。预处理后的数据随压缩包一起发送。

In [11]:
df.to_csv('data/DT_data_processed.csv',index=False)

对于预处理后的数据，我们将其从`dataframe`转换为`numpy`，然后按照一定比例，划分训练集和测试集。

In [12]:
data = df.values.astype(np.int32)
# np.random.shuffle(data)

In [13]:
split            = round(len(data) * 0.65)
train, test      = data[:split], data[split:]
X_train, Y_train = train[:, :-1], train[:, -1]
X_test, Y_test   = test[:, :-1], test[:, -1]

## 模型搭建

接下来，我们开始搭建决策树模型。本次实验中，我们采取ID3方法划分特征。

首先编写计算数据集中熵的函数。对于数据集 D ，计算熵的公式如下:

$$
\text{Entropy}(D) = -\sum_{k} p_k \log_2 p_k
$$

$p_k$ 是类别 k 在数据集 D 中的比例。

In [14]:
def entropy(y):
    unique, counts = np.unique(y, return_counts=True)
    probabilities  = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))

接下来编写寻找最佳划分属性的函数。根据ID3算法的思想，我们基于信息增益划分。

给定训练集 D 和特征 A，信息增益 $\text{Gain}(D, A)$ 定义为：

$$
\text{Gain}(D, A) = \text{Entropy}(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} \text{Entropy}(D_v)
$$


In [15]:
def ID3(X, Y):
    base_entropy   = entropy(Y)
    best_info_gain = 0
    best_feature   = -1
    n_features     = X.shape[1]
    
    for i in range(n_features):
        unique_values   = np.unique(X[:, i])
        feature_entropy = 0
        
        for value in unique_values:
            sub_Y = Y[X[:, i] == value]
            prob  = len(sub_Y) / len(Y)
            feature_entropy += prob * entropy(sub_Y)
        
        info_gain = base_entropy - feature_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature   = i
    
    return best_feature

接下来编写构建决策树的函数。就像在数据结构课程中学过的一样，我们采取递归的方法构建树即可。

In [16]:
def build(X, Y, depth=0, max_depth=5):
    if len(np.unique(Y)) == 1 or depth == max_depth:
        return np.unique(Y, return_counts=True)[0][np.argmax(np.unique(Y, return_counts=True)[1])]
    
    best_feature  = ID3(X, Y)
    tree          = {best_feature: {}}
    unique_values = np.unique(X[:, best_feature])
    
    for value in unique_values:
        sub_X     = X[X[:, best_feature] == value]
        sub_Y     = Y[X[:, best_feature] == value]
        subtree   = build(sub_X, sub_Y, depth+1, max_depth)
        tree[best_feature][value] = subtree
    
    return tree

最后编写进行预测和计算准确率的函数。

In [17]:
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree
    
    feature       = next(iter(tree))
    feature_value = sample[feature]
    
    if feature_value in tree[feature]:
        return predict(tree[feature][feature_value], sample)
    else:
        return np.random.choice(np.unique(sample))

In [18]:
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

## 实验结果

下面我们使用编写好的决策树模型进行预测。

首先我们用训练集构建决策树，看看树长什么样子。

In [19]:
tree = build(X_train,Y_train)
tree

{2: {0: {0: {0: {1: {1: 1, 2: {-1: {1: {-1: {1: 0}}}}}}, 1: 1}},
  1: {1: {0: {0: {0: {3: {0: {-1: {0: 1}}, 1: 1}},
      1: {3: {0: 1, 1: {-1: {1: 1}}}}}},
    1: {3: {0: {0: {0: 0, 1: {-1: {0: 1}}}},
      1: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 1}}}}}},
    2: {3: {0: {0: {0: 1, 1: {-1: {0: 1}}}},
      1: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 0}}}}}}}},
  2: {3: {0: {1: {0: {0: {0: {-1: {0: 0}}, 1: {-1: {0: 1}}}},
      1: {0: {0: {-1: {0: 1}}, 1: {-1: {0: 1}}}},
      2: {-1: {0: {-1: {0: 0}}}}}},
    1: {1: {0: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 1}}}},
      1: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 0}}}},
      2: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 1}}}}}}}},
  3: {1: {0: {3: {0: {0: {0: {-1: {0: 1}}, 1: {-1: {0: 1}}}},
      1: {0: {0: 1, 1: {-1: {1: 1}}}}}},
    1: {3: {0: {0: {0: {-1: {0: 1}}, 1: 1}},
      1: {0: {0: {-1: {1: 1}}, 1: {-1: {1: 0}}}}}},
    2: {0: {0: {3: {0: {-1: {0: 0}}, 1: {-1: {1: 1}}}},
      1: {3: {0: {-1: {0: 1}}, 1: {-1: {1: 1}}}}}}}},
  4: {3: {0: {0: {0: {1: {0: 

可能输出的决策树第一眼看过去不是那么易懂，这里以第一行为例简单解释一下。

```
{2: {0: {0: {0: {1: {1: 1, 2: {-1: {1: {-1: {1: 0}}}}}}, 1: 1}}...
```

- 最外层的 `{2: {...}}` 表示决策树的最顶层是基于特征索引 `2`（也就是WorkExperience） 来进行分割的。

- 内层的 `{0: {...}, 1: 1}` 表示如果特征 `2` 的值为 `0`，则无法做出决策，需要进一步检查子树 `{...}`；如果特征 `2` 的值为 `1`，则直接预测类别为 `1`，也即信誉度为好。

- 更深的层次也类似，不再赘述。

下面查看决策树在训练集上的准确率。

In [20]:
Y_pred = np.array([predict(tree,sample)for sample in X_train])
acc    = accuracy(Y_train,Y_pred)
acc

0.7769230769230769

下面查看决策树在测试集上的准确率。

In [21]:
Y_pred = np.array([predict(tree,sample)for sample in X_test])
acc    = accuracy(Y_test,Y_pred)
acc

0.7333333333333333

# 创新点

## 后剪枝操作

观察上面的结果可以发现，测试集上的准确率明显比训练集上的准确率低，说明决策树容易出现过拟合，导致泛化能力较差。

为了解决这一点，我们需要进行后剪枝操作。具体来讲，我们从树的底部开始，逐步尝试用叶节点替换子节点，并检验这种简化是否能提高测试集上的准确率，从而优化决策树的结构。

In [22]:
def cut(tree, X_val, Y_val):
    if not isinstance(tree, dict):
        return tree

    # 计算剪枝前准确率
    y_pred           = np.array([predict(tree, sample) for sample in X_val])
    initial_accuracy = np.sum(y_pred == Y_val) / len(Y_val)

    # 将当前节点转换为叶节点
    most_common      = np.bincount(Y_val).argmax()
    new_accuracy     = np.sum(Y_val == most_common) / len(Y_val)

    # 检查是否提升准确率
    if new_accuracy >= initial_accuracy:
        return most_common  
    else:
        for feature, branches in tree.items():
            for value, subtree in branches.items():
                subset_indices = X_val[:, int(feature)] == value
                if np.any(subset_indices):
                    X_sub_val  = X_val[subset_indices]
                    y_sub_val  = Y_val[subset_indices]
                    # 递归剪枝子树
                    tree[feature][value] = cut(subtree, X_sub_val, y_sub_val)
        return tree

对上面构建好的决策树，我们尝试后剪枝，并查看其后剪枝后在测试集上的表现结果。

In [23]:
tree   = cut(tree, X_test, Y_test)

In [24]:
Y_pred = np.array([predict(tree,sample)for sample in X_test])
acc    = accuracy(Y_test,Y_pred)
acc

0.8428571428571429

可以看到，测试集上的准确率明显提升，说明我们后剪枝的策略是有效的，增强了模型的泛化能力。

# 参考资料

[周志华《机器学习》——决策树基本原理](https://www.bookstack.cn/read/Vay-keen-Machine-learning-learning-notes/spilt.1.4.md)

[周志华《机器学习》——决策树剪枝处理](https://www.bookstack.cn/read/Vay-keen-Machine-learning-learning-notes/spilt.3.4.md)